In [ ]:
# LIBRARY
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss, jaccard_score, roc_auc_score, multilabel_confusion_matrix, roc_curve
from scipy.sparse import issparse
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss, jaccard_score, roc_auc_score, roc_curve
import time

In [ ]:
# DATA
df_brfss = pd.read_csv('/Users/macbook/Library/CloudStorage/GoogleDrive-nurrachmandzakiyullah@gmail.com/My Drive/23. Colab Notebooks/1. AI_Project/2. PhD_Dzaki_BRFSS/Fix_Model_HP/3. Fix Model/Final_dataset_Diabetes_Complication.csv')
x = df_brfss.iloc[:, :26].values
X = preprocessing.normalize(x)
y = df_brfss.iloc[:, 26:].values-1

In [ ]:
# Correlation matrix graph of the data set
f, ax = plt.subplots(figsize= [20,15])
sns.heatmap(df_brfss.corr(), annot=True, fmt=".2f", ax=ax, cmap = "rainbow" )
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

In [ ]:
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train=pd.DataFrame(X_train)
y_train=pd.DataFrame(y_train)
X_train=X_train.to_numpy()
y_train=y_train.to_numpy()

In [ ]:
# Decisions Tree Model
# PROBLEM TRANSFORMATION
base_classifier = RandomForestClassifier()
model = base_classifier

In [ ]:
# Get the parameter values
params = base_classifier.get_params()

# Print the parameter values
print(params)

In [ ]:
# Train with time
start = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start

In [ ]:
# 10-fold cross-validation
start = time.time()
predicted_labels = cross_val_predict(model, X, y, cv=10)
cross_val_time = time.time() - start


In [ ]:
# Testing with time
start = time.time()
predictions = model.predict(X_test)
testing_time = time.time() - start

In [ ]:
# Convert sparse matrix to dense array
predictions_dense = predictions.toarray()

# 1. Matrix Evaluation Label Based


In [ ]:
# Multilabel confusion matrix
cm = multilabel_confusion_matrix(y_test, predictions_dense)

# Determine the grid dimensions for subplots
num_labels = cm.shape[0]
cols = min(num_labels, 2)
rows = (num_labels + cols - 1) // cols

# Plot confusion matrix
fig, axes = plt.subplots(rows, cols, figsize=(12, 8))
for i, (matrix, ax) in enumerate(zip(cm, axes.ravel())):
    sns.heatmap(matrix, annot=True, fmt="d", cbar=False, cmap='Blues', ax=ax)
    ax.set_title(f"Confusion Matrix - Label {i}")
    ax.set_xlabel("Predicted Labels")
    ax.set_ylabel("True Labels")

# Remove unused subplots if necessary
if num_labels < rows * cols:
    for j in range(num_labels, rows * cols):
        fig.delaxes(axes.ravel()[j])

plt.tight_layout()
plt.show()

In [ ]:
# Initialize the roc_auc_per_label dictionary
roc_auc_per_label = {}

# Plot ROC curve for each label
plt.figure(figsize=(10, 6))
for i in range(y.shape[1]):
    y_test_label = np.squeeze(y_test[:, i].toarray()) if issparse(y_test[:, i]) else np.squeeze(y_test[:, i])
    y_pred_label = np.squeeze(predictions_dense[:, i].toarray()) if issparse(predictions_dense[:, i]) else np.squeeze(predictions_dense[:, i])
    fpr, tpr, _ = roc_curve(y_test_label, y_pred_label)
    roc_auc_per_label[i] = roc_auc_score(y_test_label, y_pred_label)
    plt.plot(fpr, tpr, label='Label %d (AUC = %0.2f)' % (i, roc_auc_per_label[i]))

# Plot aggregate AUC ROC
fpr_aggregate, tpr_aggregate, _ = roc_curve(y_test.ravel(), predictions_dense.ravel())
roc_auc_aggregate = roc_auc_score(y_test.ravel(), predictions_dense.ravel())
plt.plot(fpr_aggregate, tpr_aggregate, label='Aggregate ROC Curve (AUC = %0.2f)' % roc_auc_aggregate)

# Plot diagonal line
plt.plot([0, 1], [0, 1], 'k--')

# Set plot limits and labels
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Initialize the roc_auc_per_label dictionary
roc_auc_per_label = {}

# Plot ROC curve for each label
plt.figure(figsize=(10, 6))
for i in range(y.shape[1]):
    y_test_label = np.array(y_test[:, i].toarray().ravel()) if issparse(y_test[:, i]) else np.array(y_test[:, i])
    y_pred_label = np.array(predictions_dense[:, i].toarray().ravel()) if issparse(predictions_dense[:, i]) else np.array(predictions_dense[:, i])
    fpr, tpr, _ = roc_curve(y_test_label, y_pred_label)
    roc_auc_per_label[i] = roc_auc_score(y_test_label, y_pred_label)
    plt.plot(fpr, tpr, label='Label %d (AUC = %0.2f)' % (i, roc_auc_per_label[i]))

# Plot aggregate AUC ROC
fpr_aggregate, tpr_aggregate, _ = roc_curve(y_test.ravel(), predictions_dense.ravel())
roc_auc_aggregate = roc_auc_score(y_test.ravel(), predictions_dense.ravel())
plt.plot(fpr_aggregate, tpr_aggregate, label='Aggregate ROC Curve (AUC = %0.2f)' % roc_auc_aggregate)

# Plot diagonal line
plt.plot([0, 1], [0, 1], 'k--')

# Set plot limits and labels
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.show()


# 2. Matrix Evaluation Example Based


In [ ]:
# Evaluation metrics
accuracy = accuracy_score(y_test, predictions_dense)
precision = precision_score(y_test, predictions_dense, average='micro')
recall = recall_score(y_test, predictions_dense, average='micro')
f1 = f1_score(y_test, predictions_dense, average='micro')
hamming_loss_score = hamming_loss(y_test, predictions_dense)
jaccard = jaccard_score(y_test, predictions_dense, average='micro')
auc_roc = roc_auc_score(y_test, predictions_dense)

In [ ]:
# Print results
print('Training Time:', round(training_time, 2), 'seconds')
print('Cross-validated Testing Time:', round(cross_val_time, 2), 'seconds')
print('Testing Time on Specific Test Set:', round(testing_time, 2), 'seconds')
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('Hamming Loss:', hamming_loss_score)
print('Jaccard Score:', jaccard)
print('AUC-ROC:', auc_roc)

In [ ]:
import sklearn.metrics

print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_test, predictions_dense, normalize=True, sample_weight=None)))
print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_test, predictions_dense))) 
print('Recall: {0}'.format(sklearn.metrics.precision_score(y_true=y_test, y_pred=predictions_dense, average='samples'))) 
print('Precision: {0}'.format(sklearn.metrics.recall_score(y_true=y_test, y_pred=predictions_dense, average='samples')))
print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_true=y_test, y_pred=predictions_dense, average='samples'))) 

In [ ]:
#Subet Accuracy atau Exact Match Ratio
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

In [ ]:
# HITUNG HAMMING LOSS
from sklearn.metrics import hamming_loss
hamming_loss(y_test, predictions)

In [ ]:
from sklearn.metrics import jaccard_score
print('jaccard_score_Samples: {0}'.format(jaccard_score(y_test, predictions, average='samples')))
print('jaccard_score_Micro: {0}'.format(jaccard_score(y_test, predictions, average="micro")))
print('jaccard_score_Macro: {0}'.format(jaccard_score(y_test, predictions, average="macro")))


In [ ]:
from sklearn.metrics import zero_one_loss
print('Zero_one_loss: {0}'.format(zero_one_loss(y_test, predictions)))
print('Zero_one_loss_Samples: {0}'.format(zero_one_loss(y_test, predictions, normalize=False)))

In [ ]:
from sklearn.metrics import f1_score

print('f1_score_micro: {0}'.format(f1_score(y_test, predictions, average='micro')))
print('f1_score_macro: {0}'.format(f1_score(y_test, predictions, average='macro')))
print('f1_score_weighted: {0}'.format(f1_score(y_test, predictions, average='weighted')))
print('f1_score_none: {0}'.format(f1_score(y_test, predictions, average=None)))


In [ ]:
from sklearn.metrics import fbeta_score

print('fbeta_macro: {0}'.format(fbeta_score(y_test, predictions, average='macro', beta=0.5)))
print('fbeta_micro: {0}'.format(fbeta_score(y_test, predictions, average='micro', beta=0.5)))
print('fbeta_weighted: {0}'.format(fbeta_score(y_test, predictions, average='weighted', beta=0.5)))
print('fbeta_none: {0}'.format(fbeta_score(y_test, predictions, average=None, beta=0.5)))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

print('recision_recall_fscore_support_macro: {0}'.format(precision_recall_fscore_support(y_test, predictions, average='macro')))
print('recision_recall_fscore_support_micro: {0}'.format(precision_recall_fscore_support(y_test, predictions, average='micro')))
print('recision_recall_fscore_support_weighted: {0}'.format(precision_recall_fscore_support(y_test, predictions, average='weighted')))
print('recision_recall_fscore_support_none: {0}'.format(precision_recall_fscore_support(y_test, predictions, average=None)))

In [ ]:
from sklearn.metrics import precision_score
print('precision_score_macro: {0}'.format(precision_score(y_test, predictions, average='macro')))
print('precision_score_micro: {0}'.format(precision_score(y_test, predictions, average='micro')))
print('precision_score_weighted: {0}'.format(precision_score(y_test, predictions, average='weighted')))
print('precision_score_none: {0}'.format(precision_score(y_test, predictions, average=None)))
print('precision_score_zero_divisio: {0}'.format(precision_score(y_test, predictions, average=None, zero_division=1)))


In [ ]:
from sklearn.metrics import recall_score
print('recall_score_macro: {0}'.format(recall_score(y_test, predictions, average='macro')))
print('recall_score_micro: {0}'.format(recall_score(y_test, predictions, average='micro')))
print('recall_score_weighted: {0}'.format(recall_score(y_test, predictions, average='weighted')))
print('recall_score_none: {0}'.format(recall_score(y_test, predictions, average=None)))
print('recall_score_zero_divisio: {0}'.format(recall_score(y_test, predictions, average=None, zero_division=1)))


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, predictions)

In [ ]:
# CLASSIFICATION REPORT : precision, recacll f1-score support every label
from sklearn.metrics import classification_report 
print(classification_report(y_test, predictions))

In [ ]:
print("Actual \n", y_test)
print("\nPredicted \n", predictions)